In [108]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [109]:
deliveries = pd.read_csv("../Data/deliveries_cleaned.csv")
matches = pd.read_csv("../Data/matches_cleaned.csv")

In [110]:
deliveries.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,dismissal_type
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,NaN
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,NaN
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,NaN
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,NaN
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,NaN


In [111]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               950 non-null    int64  
 1   City             950 non-null    object 
 2   Date             950 non-null    object 
 3   Season           950 non-null    object 
 4   MatchNumber      950 non-null    object 
 5   Team1            950 non-null    object 
 6   Team2            950 non-null    object 
 7   Venue            950 non-null    object 
 8   TossWinner       950 non-null    object 
 9   TossDecision     950 non-null    object 
 10  SuperOver        946 non-null    object 
 11  WinningTeam      946 non-null    object 
 12  WonBy            950 non-null    object 
 13  Margin           932 non-null    float64
 14  method           19 non-null     object 
 15  Player_of_Match  946 non-null    object 
 16  Team1Players     950 non-null    object 
 17  Team2Players    

Batting team points

In [112]:
def BattingTeamFinder(x):
    if x['Team1'] == x['TossWinner']:
        return x['Team1'] if x['TossDecision'] == 'bat' else x['Team2']
    else:
        return x['Team2'] if x['TossDecision'] == 'bat' else x['Team1']

In [113]:
matches['BattingTeam'] = matches[['Team1', 'Team2', 'TossWinner', 'TossDecision']].apply(lambda x: BattingTeamFinder(x), axis=1)

In [114]:
matches['batting_team_points'] = matches[['Season', 'ID', 'Date', 'BattingTeam']].apply(lambda x: (matches[(matches['Season'] == x['Season']) & (matches['Date'] < x['Date']) & (matches['WinningTeam'] == x['BattingTeam'])].shape[0] * 2), axis = 1)

Bowling Team Points

In [115]:
def BowlingTeamFinder(x):
    if x['Team1'] == x['TossWinner']:
        return x['Team1'] if x['TossDecision'] == 'field' else x['Team2']
    else:
        return x['Team2'] if x['TossDecision'] == 'field' else x['Team1']

In [116]:
matches['BowlingTeam'] = matches[['Team1', 'Team2', 'TossWinner', 'TossDecision']].apply(lambda x: BowlingTeamFinder(x), axis = 1)

In [117]:
matches['bowling_team_points'] = matches[['Season', 'ID', 'Date', 'BowlingTeam']].apply(lambda x: (matches[(matches['Season'] == x['Season']) & (matches['Date'] < x['Date']) & (matches['WinningTeam'] == x['BowlingTeam'])].shape[0] * 2), axis = 1)

Home ground advantage for bowling team

In [118]:
home_grounds = pd.read_csv('../Data/home ground.csv')

In [119]:
home_grounds.loc[home_grounds['team1'] == 'Gujarat Lions', 'team1'] = 'Gujarat Titans'
home_grounds.loc[home_grounds['team1'] == 'Kings XI Punjab', 'team1'] = 'Punjab Kings'
home_grounds.iloc[-1] = ['Lucknow Super Giants', 'Lucknow']

In [120]:
def groundAdvCalculator(x):
  adv = 0
  adv += 1 if x['City'] == home_grounds[home_grounds['team1'] == x['BowlingTeam']]['city'].iloc[0] else 0
  adv -= 1 if x['City'] == home_grounds[home_grounds['team1'] == x['BattingTeam']]['city'].iloc[0] else 0
  return adv

In [121]:
matches['ground_adv'] = matches.apply(lambda x: groundAdvCalculator(x), axis = 1)

In [122]:
matches.drop(['BattingTeam', 'BowlingTeam'], axis=1, inplace=True)

In [123]:
df = pd.merge(deliveries, matches, 'inner', on='ID')
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'dismissal_type', 'City', 'Date',
       'Season', 'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner',
       'TossDecision', 'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'batting_team_points', 'bowling_team_points', 'ground_adv'],
      dtype='object')

In [124]:
df.drop(df[df['WonBy'] == 'SuperOver'].index, inplace=True)

In [125]:
df.drop(df[df['method'] == 'D/L'].index, inplace=True)

In [126]:
df.loc[(df['TossDecision'] == 'bat') & (df['TossWinner'] == df['BattingTeam']), 'innings'] = 1
df.loc[(df['TossDecision'] == 'bowl') & (df['TossWinner'] != df['BattingTeam']), 'innings'] = 1
df.loc[(df['TossDecision'] == 'bat') & (df['TossWinner'] != df['BattingTeam']), 'innings'] = 2
df.loc[(df['TossDecision'] == 'bowl') & (df['TossWinner'] == df['BattingTeam']), 'innings'] = 2

In [127]:
df['innings'].unique()

array([1, 2], dtype=int64)

In [128]:
inn2 = df[df["innings"] == 2]

In [129]:
inn2.drop(['extra_type', 'batsman_run', 'extras_run', 'non_boundary', 'SuperOver', 'method', 'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2'], axis = 1, inplace = True)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\201460119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2.drop(['extra_type', 'batsman_run', 'extras_run', 'non_boundary', 'SuperOver', 'method', 'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2'], axis = 1, inplace = True)


**Features to be extracted for winning prediction**  
Runs Scored   
Target  
Run rate   
Required Run Rate   
Points scored by batting team in the season  
Points scored by bowling team in the season  
Home Ground advantage for bowling team?    
Wicket remaining in hand  
Bowling team won?  
No of boundaries scored  

Runs Scored

In [130]:
inn2['runs scored'] = inn2[['ID', 'overs', 'ballnumber', 'total_run']].apply(lambda x: inn2[(inn2['ID'] == x.ID) & (inn2['overs'] <= x.overs) & (inn2['ballnumber'] <= x.ballnumber)]['total_run'].sum() + 0, axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\1827609532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['runs scored'] = inn2[['ID', 'overs', 'ballnumber', 'total_run']].apply(lambda x: inn2[(inn2['ID'] == x.ID) & (inn2['overs'] <= x.overs) & (inn2['ballnumber'] <= x.ballnumber)]['total_run'].sum() + 0, axis = 1)


Target

In [31]:
inn1 = pd.read_csv("../Data/inning1.csv")

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\77058236.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  inn1 = pd.read_csv("../Data/inning1.csv")


In [36]:
inn1['runs scored'] = inn1[['ID', 'overs', 'ballnumber', 'total_run']].apply(lambda x: inn1[(inn1['ID'] == x.ID) & (inn1['overs'] <= x.overs) & (inn1['ballnumber'] <= x.ballnumber)]['total_run'].sum() + 0, axis = 1)

In [131]:
inn1['ID'] = pd.to_numeric(inn1['ID'])

In [132]:
def target_extractor(id, inn1):
  try:
    match = inn1[inn1['ID'] == id]
    match.sort_values(["overs", "ballnumber"])
    return match.tail(1)['runs scored'].iloc[0]
  except:
    return float('nan')

In [133]:
inn2['target'] = inn2['ID'].apply(lambda x: target_extractor(x, inn1))

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\3635598011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['target'] = inn2['ID'].apply(lambda x: target_extractor(x, inn1))


Run Rate Calculation

In [134]:
inn2['runrate'] = inn2[['runs scored', 'overs', 'ballnumber']].apply(lambda x: x['runs scored']/(x['overs'] + (x['ballnumber']/6)), axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\1697964358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['runrate'] = inn2[['runs scored', 'overs', 'ballnumber']].apply(lambda x: x['runs scored']/(x['overs'] + (x['ballnumber']/6)), axis = 1)


Required Run rate

In [135]:
inn2['req runrate'] = inn2[['runs scored', 'overs', 'ballnumber', 'target']].apply(lambda x: (x['target'] - x['runs scored'])/((20 - x['overs']) + ((6 - x['ballnumber'])/6)), axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\893587982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['req runrate'] = inn2[['runs scored', 'overs', 'ballnumber', 'target']].apply(lambda x: (x['target'] - x['runs scored'])/((20 - x['overs']) + ((6 - x['ballnumber'])/6)), axis = 1)


In [140]:
inn2['wickets_gone'] = inn2.apply(lambda x: inn2[(inn2['ID'] == x.ID) & (inn2['overs'] <= x.overs) & (inn2['ballnumber'] <= x.ballnumber)]['isWicketDelivery'].sum(), axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\2876640830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['wickets_gone'] = inn2.apply(lambda x: inn2[(inn2['ID'] == x.ID) & (inn2['overs'] <= x.overs) & (inn2['ballnumber'] <= x.ballnumber)]['isWicketDelivery'].sum(), axis = 1)


In [145]:
inn2['BowlingTeam'] = inn2[['Team1', 'Team2', 'TossWinner', 'TossDecision']].apply(lambda x: BowlingTeamFinder(x), axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\3851908878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['BowlingTeam'] = inn2[['Team1', 'Team2', 'TossWinner', 'TossDecision']].apply(lambda x: BowlingTeamFinder(x), axis = 1)


In [146]:
inn2['bowling_team_won'] = inn2.apply(lambda x: 1 if x['BowlingTeam'] == x['WinningTeam'] else 0, axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\504546653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['bowling_team_won'] = inn2.apply(lambda x: 1 if x['BowlingTeam'] == x['WinningTeam'] else 0, axis = 1)


In [147]:
inn2['boundaries_scored'] = inn2.apply(lambda x: inn2[(inn2.ID == x.ID) & (inn2.overs <= x.overs) & (inn2.ballnumber <= x.ballnumber) & (inn2['total_run'] > 3)]['total_run'].count(), axis = 1)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\2499875677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['boundaries_scored'] = inn2.apply(lambda x: inn2[(inn2.ID == x.ID) & (inn2.overs <= x.overs) & (inn2.ballnumber <= x.ballnumber) & (inn2['total_run'] > 3)]['total_run'].count(), axis = 1)


In [149]:
inn2.to_csv('../Data/final_inning2.csv', index=False)

In [150]:
inn2.drop(['ID', 'innings', 'batter', 'bowler', 'non-striker', 'player_out', 'kind', 'fielders_involved', 'dismissal_type', 'City', 'Date', 'MatchNumber', 'Team1', 'Team2', 'Venue', 'WinningTeam', 'WonBy', 'Margin'], axis = 1, inplace = True)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\3129533939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2.drop(['ID', 'innings', 'batter', 'bowler', 'non-striker', 'player_out', 'kind', 'fielders_involved', 'dismissal_type', 'City', 'Date', 'MatchNumber', 'Team1', 'Team2', 'Venue', 'WinningTeam', 'WonBy', 'Margin'], axis = 1, inplace = True)


In [151]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [152]:
le = LabelEncoder()

In [154]:
le.fit(inn2['BattingTeam'])

LabelEncoder()

In [155]:
inn2['BattingTeam'] = le.transform(inn2['BattingTeam'])

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\4080119729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['BattingTeam'] = le.transform(inn2['BattingTeam'])


In [156]:
inn2['BowlingTeam'] = le.transform(inn2['BowlingTeam'])

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\3589448604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['BowlingTeam'] = le.transform(inn2['BowlingTeam'])


In [157]:
inn2.drop('Season', axis = 1, inplace = True)

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\447493586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2.drop('Season', axis = 1, inplace = True)


In [158]:
inn2['TossWinner'] = le.transform(inn2['TossWinner'])

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\53841552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['TossWinner'] = le.transform(inn2['TossWinner'])


In [160]:
le.fit(inn2['TossDecision'])

LabelEncoder()

In [161]:
inn2['TossDecision'] = le.transform(inn2['TossDecision'])

C:\Users\siddr\AppData\Local\Temp\ipykernel_18208\3183679971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inn2['TossDecision'] = le.transform(inn2['TossDecision'])


In [162]:
inn2.head()

,overs,ballnumber,total_run,isWicketDelivery,BattingTeam,TossWinner,TossDecision,batting_team_points,bowling_team_points,ground_adv,runs scored,target,runrate,req runrate,wickets_gone,BowlingTeam,bowling_team_won,boundaries_scored
120,0,1,0,0,2,8,0,20,22,0,0,130,0.0,6.240000,0,2,1,0
121,0,2,0,0,2,8,0,20,22,0,0,130,0.0,6.290323,0,2,1,0
122,0,3,1,0,2,8,0,20,22,0,1,130,2.0,6.292683,0,2,1,0
123,0,4,0,0,2,8,0,20,22,0,1,130,1.5,6.344262,0,2,1,0
124,0,5,0,0,2,8,0,20,22,0,1,130,1.2,6.396694,0,2,1,0


In [164]:
inn2.to_csv("../Data/inning2.csv", index=False)